In [1]:
import pandas as pd
from sklearn import metrics
import matplotlib.pyplot as plt

In [2]:
main_data = pd.read_csv('C:/Users/13903/Desktop/final_main.csv')
duplicate_data = pd.read_csv('C:/Users/13903/Desktop/duplicate.csv')
main_data

,ID,Last Name,First Name,Date of Birth,Mother's Birth Last Name,Indigenous Identity,Religion,Ethnicity,Race,Place of Birth,Citizenship,Language (Mother Tongue),cluster_id
0,5d322502,Miller,Marian,04/08/2009,NaN,NaN,NaN,Black,Black,Canada,Canada,Spanish,7383.0
1,cb90c9e8,Jackson,James,09/17/2021,NaN,Native Hawaiian,NaN,Native Hawaiian,Native Hawaiian,NaN,Canada,English,1027.0
2,ed68be4d,Tolbert,Steven,01/13/2008,NaN,Indian,NaN,Indian,Indian,Canada,Canada,NaN,1026.0
3,e73ea697,Boling,Dorothy,05/02/2012,NaN,NaN,Jewish,NaN,White,NaN,NaN,NaN,1025.0
4,24d31a35,Yerian,Elwood,08/18/2019,NaN,NaN,Islam,NaN,Middle Easterner,NaN,Canada,English,8465.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19152,56496712,Crutcher,Alicia,11/17/2016,NaN,NaN,Other Christianity,NaN,Southeast Asian,Canada,Canada,NaN,20295.0
19153,a801805e,Morris,Johanna,01/22/2014,Morice,NaN,NaN,NaN,Middle Easterner,NaN,NaN,English,20296.0
19154,df377f5f,Mann,Lena,10/14/2018,NaN,Pacific Islander,NaN,Pacific Islander,Pacific Islander,NaN,Canada,NaN,20297.0
19155,d4f04ec2,Rolling,Virginia,04/02/2021,NaN,American Indian,NaN,American Indian,American Indian,NaN,Canada,NaN,20298.0


In [3]:
main_data.isna().sum()

ID                              0
Last Name                       0
First Name                      0
Date of Birth                   0
Mother's Birth Last Name    18304
Indigenous Identity         10392
Religion                    13836
Ethnicity                    7294
Race                            0
Place of Birth              11437
Citizenship                  3316
Language (Mother Tongue)     8658
cluster_id                      0
dtype: int64

drop the variables which amount of NA is larger than 10000 (50%)

insight --- the contents of indigenous identity, ethnicity and race are the same.

In [4]:
data = main_data.drop(main_data.columns[[4,5,6,7,9]], axis=1)
data.fillna(value='None', inplace = True)
data.columns = ['ID', 'Last_Name', 'First_Name', 'Date_of_Birth', 'Race', 'Citizenship','Language_Mother_Tongue', 'cluster_id']
data.isna().sum()

ID                        0
Last_Name                 0
First_Name                0
Date_of_Birth             0
Race                      0
Citizenship               0
Language_Mother_Tongue    0
cluster_id                0
dtype: int64

In [5]:
# feature engineering on date
data['Date_of_Birth'] = pd.to_datetime(data['Date_of_Birth'], format = '%m/%d/%Y')
data['year'] = data['Date_of_Birth'].dt.year
data['month'] = data['Date_of_Birth'].dt.month
data['day'] = data['Date_of_Birth'].dt.day
#data.drop(columns=['Date_of_Birth'], inplace = True)
data.drop(columns=['Date_of_Birth'], inplace = True)
data.isna().sum()


ID                        0
Last_Name                 0
First_Name                0
Race                      0
Citizenship               0
Language_Mother_Tongue    0
cluster_id                0
year                      0
month                     0
day                       0
dtype: int64

In [6]:
data.dtypes

ID                         object
Last_Name                  object
First_Name                 object
Race                       object
Citizenship                object
Language_Mother_Tongue     object
cluster_id                float64
year                        int64
month                       int64
day                         int64
dtype: object

In [7]:
# combination all columns into one dataframe with 1 column 
# so that it is able to apply bagging of words
row_input = data.columns.tolist()

word_gallery = pd.DataFrame(data['ID'] + ',' + data['Last_Name'] + ',' + data['First_Name']\
                            + ',' + data['year'].astype(str) + ',' +\
                                data['month'].astype(str) + ',' + data['day'].astype(str) + ','+\
                                    data['Race'] + ','+ data['Citizenship'] + ',' \
                                    + data['Language_Mother_Tongue'], columns=['BoW'])
word_gallery


,BoW
0,"5d322502,Miller,Marian,2009,4,8,Black,Canada,S..."
1,"cb90c9e8,Jackson,James,2021,9,17,Native Hawaii..."
2,"ed68be4d,Tolbert,Steven,2008,1,13,Indian,Canad..."
3,"e73ea697,Boling,Dorothy,2012,5,2,White,None,None"
4,"24d31a35,Yerian,Elwood,2019,8,18,Middle Easter..."
...,...
19152,"56496712,Crutcher,Alicia,2016,11,17,Southeast ..."
19153,"a801805e,Morris,Johanna,2014,1,22,Middle Easte..."
19154,"df377f5f,Mann,Lena,2018,10,14,Pacific Islander..."
19155,"d4f04ec2,Rolling,Virginia,2021,4,2,American In..."


assign train test set

In [8]:
y_train = data['cluster_id']
X_train = word_gallery

In [9]:
# build bag of words
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

# fit the bag of words model

X_train_count = vectorizer.fit_transform(X_train['BoW'])
X_train_count.shape

(19157, 29589)

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_count)
X_train_tfidf.shape

(19157, 29589)

In [11]:
# data preprocessing on duplicate data

duplicate_data

,ID,Last Name,First Name,Date of Birth,Mother's Birth Last Name,Indigenous Identity,Religion,Ethnicity,Race,Place of Birth,Citizenship,Language (Mother Tongue),cluster_id
0,73c2ab6a,NaN,Steven,01/13/2008,NaN,NaN,NaN,Indian,NaN,Canada,NaN,NaN,1026
1,488f1063,NaN,Dorothy,05/02/2012,NaN,NaN,Jewish,NaN,NaN,NaN,NaN,NaN,1025
2,331bea71,Yerian,Ellwood,08/18/2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,English,8465
3,ac3aaf9c,NaN,Robin,08/25/2013,NaN,NaN,NaN,Indian,NaN,Canada,NaN,English,1024
4,ae9c2ae4,NaN,Raymond,05/25/2017,NaN,NaN,Other Christianity,NaN,NaN,Canada,NaN,English,1023
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9256,08606dd8,Avila,NaN,06/24/2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1357
9257,8507e3f4,Fernandez,NaN,08/07/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1356
9258,d5285223,Mathews,NaN,02/28/2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7859
9259,7157d300,Konty,NaN,12/15/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1355


In [12]:
ddata = duplicate_data.drop(duplicate_data.columns[[4,5,6,7,9,12]], axis=1)
ddata.fillna(value='None', inplace = True)
ddata.columns = ['ID', 'Last_Name', 'First_Name', 'Date_of_Birth', 'Race', 'Citizenship',\
                 'Language_Mother_Tongue']
ddata.isna().sum()

ID                        0
Last_Name                 0
First_Name                0
Date_of_Birth             0
Race                      0
Citizenship               0
Language_Mother_Tongue    0
dtype: int64

In [13]:
# feature engineering on date
ddata['Date_of_Birth'] = pd.to_datetime(ddata['Date_of_Birth'], format = '%m/%d/%Y')
ddata['year'] = ddata['Date_of_Birth'].dt.year
ddata['month'] = ddata['Date_of_Birth'].dt.month
ddata['day'] = ddata['Date_of_Birth'].dt.day
ddata.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9261 entries, 0 to 9260
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID                      9261 non-null   object        
 1   Last_Name               9261 non-null   object        
 2   First_Name              9261 non-null   object        
 3   Date_of_Birth           9261 non-null   datetime64[ns]
 4   Race                    9261 non-null   object        
 5   Citizenship             9261 non-null   object        
 6   Language_Mother_Tongue  9261 non-null   object        
 7   year                    9261 non-null   int64         
 8   month                   9261 non-null   int64         
 9   day                     9261 non-null   int64         
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 723.6+ KB


In [14]:
word_bag = pd.DataFrame(ddata['ID'].astype(str) + ',' + ddata['Last_Name'].astype(str) +\
                              ',' + ddata['First_Name'].astype(str) +\
                              ',' + ddata['year'].astype(str) + ',' + ddata['month'].astype(str) +\
                                  ',' + ddata['day'].astype(str) + ','+\
                                      ddata['Race'].astype(str) + ','+\
                                      ddata['Citizenship'].astype(str) + ',' +\
                                          ddata['Language_Mother_Tongue'].astype(str),\
                                          columns=['BoW'])

In [15]:
X_test_count = vectorizer.transform(word_bag['BoW'])
X_test_count.shape
X_test_tfidf = tfidf_transformer.transform(X_test_count)
X_test_tfidf.shape

(9261, 29589)

In [16]:
duplicate_data.sort_values(['cluster_id'])

,ID,Last Name,First Name,Date of Birth,Mother's Birth Last Name,Indigenous Identity,Religion,Ethnicity,Race,Place of Birth,Citizenship,Language (Mother Tongue),cluster_id
1551,db41c6a1,NaN,Camilla,04/29/2022,NaN,NaN,NaN,Native Hawaiian,NaN,NaN,NaN,NaN,0
1550,dde408f0,NaN,Yolanda,05/15/2018,NaN,NaN,NaN,NaN,NaN,Canada,NaN,Tagalog,1
1549,4e82ee4a,NaN,Marion,08/28/2022,NaN,NaN,Pentecostalism and other Charismatic,NaN,NaN,Canada,NaN,Portuguese,2
1547,3eef0d37,NaN,Jesse,06/13/2016,NaN,NaN,NaN,Middle Easterner,NaN,Canada,NaN,NaN,3
1546,b2bd862f,NaN,Jean,07/09/2012,NaN,NaN,Judaism,NaN,NaN,NaN,NaN,Tagalog,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2204,c76ffc85,Ripley,Diahann,07/19/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9256
2132,8e1e86e4,Olaughlin,Rueth,06/27/2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9257
2079,d22930bf,Briggs,Cheri,09/29/2007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,English,9258
1868,462f4f66,Dillon,Rueth,03/23/2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9259


In [17]:
y_test = duplicate_data['cluster_id']

In [19]:
# Naive bayes
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB().fit(X_train_tfidf, y_train)
pred1 = mnb.predict(X_test_tfidf)
accuracy1 = metrics.accuracy_score(y_test, pred1)
print(accuracy1)

0.8372745923766332


In [20]:
mnb = MultinomialNB(alpha = 0.1).fit(X_train_tfidf, y_train)
pred1 = mnb.predict(X_test_tfidf)
accuracy1 = metrics.accuracy_score(y_test, pred1)
print(accuracy1)

0.842673577367455


In [34]:
mnb = MultinomialNB(alpha = 2).fit(X_train_tfidf, y_train)
pred1 = mnb.predict(X_test_tfidf)
accuracy1 = metrics.accuracy_score(y_test, pred1)
print(accuracy1)

0.8345750998812224


In [35]:
mnb = MultinomialNB(alpha = 0.01).fit(X_train_tfidf, y_train)
pred1 = mnb.predict(X_test_tfidf)
accuracy1 = metrics.accuracy_score(y_test, pred1)
print(accuracy1)

0.8312277291869129
